In [1]:
# from __future__ import print_function
# from keras.models import Sequential
# from keras.layers.core import Dense, Activation, Dropout
# from keras.layers.recurrent import LSTM
# from keras.datasets.data_utils import get_file
# from keras.callbacks import Callback
import numpy as np
import random
import sys

import numpy as np
import csv
import matplotlib.pyplot as plt

synthetic_data_set = "syntheticDetailed/naive_c5_q50_s4000_v0.csv"

In [2]:
# Read in the data set
# This function can be moved to utils.py
data_array = np.array(list(csv.reader(open(synthetic_data_set,"rb"),delimiter=','))).astype('int')
print (data_array[0:10])
data_array = data_array[:1000]
num_samples = data_array.shape[0]
num_problems = data_array.shape[1]

# time steps is number of problems - 1 because we cannot predict on the last problem.
num_timesteps = num_problems - 1 
# Split data into train and test (half and half)
train = data_array[0:num_samples/2,:]
test = data_array[num_samples/2:num_samples,:]
print (train.shape)
print (test.shape)

num_train = train.shape[0]
num_test = test.shape[0]


print('Vectorization...')
X_train = np.zeros((num_train, num_timesteps, num_problems * 2), dtype=np.bool)
y_train = np.zeros((num_train, num_timesteps), dtype=np.int)

# Create 3-dimensional input tensor with one-hot encodings for each sample
# the dimension of each vector for a student i and time t is 2 * num_problems
# where the first half corresponds to the correctly answered problems and the
# second half to the incorrectly answered ones.
for i in xrange(num_train):
    
    # for the first time step. Done separately so we can populate the output 
    # tensor at the same time, which is shifted back by 1.

    for t in xrange(0,num_timesteps):
        p = t # since timestep t corresponds to problem p where t=p
        if train[i,p] == 1:
            X_train[i, t, p] = 1 
        else:
            X_train[i, t, num_problems + p] = 1
        # this is a special case for the synthetic data set, where the next problem 
        # is just the current problem index + 1
        y_train[i,t] = p + 1
correctness = train

print ("Vectorization done!")


[[0 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0
  0 1 1 0 1 1 0 1 0 1 1 0 1]
 [1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
  1 0 0 0 1 0 1 0 0 1 0 0 0]
 [0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 1
  1 1 1 0 0 1 1 1 0 0 1 1 1]
 [0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 0 0 1 1
  0 1 0 0 1 1 1 1 0 0 1 1 1]
 [0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1
  1 1 1 0 0 0 1 0 1 1 1 1 1]
 [0 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1
  1 1 0 0 0 0 1 0 0 0 1 1 1]
 [0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1
  1 1 1 0 0 1 1 1 1 1 1 1 1]
 [0 1 0 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 0 0
  1 0 1 0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1
  1 0 1 0 0 0 1 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1

In [3]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
learning_rate = 1e-1
epochs = 50

# model parameters
Wxh = np.random.randn(hidden_size, num_problems * 2)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(num_problems, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((num_problems, 1)) # output bias

def lossFun(inputs, targets, correctness, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps, ps_denom = {}, {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in xrange(len(inputs)):
        xs[t] = inputs[t,:].reshape((num_problems * 2, 1))
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps_denom[t] = np.sum(np.exp(ys[t]))
        ps[t] = np.exp(ys[t]) / ps_denom[t] # probabilities for next chars

        # softmax (cross-entropy loss)
        if correctness[targets[t]] == 1:
            loss += -np.log(ps[t][targets[t],0]) 
        else:
            loss += -np.log(1-ps[t][targets[t],0]) 
        # backward pass: compute gradients going backwards
        dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
        dbh, dby = np.zeros_like(bh), np.zeros_like(by)
        dhnext = np.zeros_like(hs[0])


    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
        if correctness[targets[t]] == 1:
            dy[targets[t]] -= 1 # backprop into y
        else:
            for p in xrange(num_problems):
                if p != targets[t]:
                    dy[p] -= np.exp(ys[t][p]) / (ps_denom[t] - np.exp(ys[t][targets[t]]))



        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]




In [ ]:
def accuracy(ps, targets, correctness):
    """
    Computes the accuracy using the predictions at each time step.
    For each t, if probability of next problem is > 0.5 for correct, or <= 0.5 
    for incorrect, then count this as correct prediction.
    """
    num_correct = 0
    for t in xrange(num_timesteps):
        predicted_prob = ps[t][targets[t],0] 
        if (predicted_prob >= 0.5 and correctness[targets[t]] == 1) or (predicted_prob < 0.5 and correctness[targets[t]] == 0):
            num_correct += 1
    accuracy = num_correct / float(num_timesteps)
    return accuracy


# def sample(h, seed_ix, n):
#   """ 
#   sample a sequence of integers from the model 
#   h is memory state, seed_ix is seed letter for first time step
#   """
#   x = np.zeros((num_problems, 1))
#   x[seed_ix] = 1
#   ixes = []
#   for t in xrange(n):
#     h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
#     y = np.dot(Why, h) + by
#     p = np.exp(y) / np.sum(np.exp(y))
#     ix = np.random.choice(range(num_problems), p=p.ravel())
#     x = np.zeros((num_problems, 1))
#     x[ix] = 1
#     ixes.append(ix)
#   return ixes

def forward_pass(inputs):
    xs, hs, ys, ps, ps_denom = {}, {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    for t in xrange(len(inputs)):
        xs[t] = inputs[t,:].reshape((num_problems * 2, 1))
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps_denom[t] = np.sum(np.exp(ys[t]))
        ps[t] = np.exp(ys[t]) / ps_denom[t] # probabilities for next chars
    return ps

In [ ]:
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/num_problems)*num_timesteps # loss at iteration 0

losses = []
for e in xrange(epochs):
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    total_acc = 0.0
    for i in xrange(num_train):
        inputs = X_train[i,:,:].reshape((num_timesteps, num_problems * 2))
        targets = y_train[i,:].reshape((num_timesteps,))
        correctness_for_student = correctness[i,:].reshape((num_problems))

        # sample from the model now and then
        # if n % 100 == 0:
        #   sample_ix = sample(hprev, inputs[0], 200)
        #   txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        #   print '----\n %s \n----' % (txt, )

        # forward num_timesteps characters through the net and fetch gradient
        loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, correctness_for_student, hprev)
        #       smooth_loss = smooth_loss * 0.999 + loss * 0.001
        losses.append(loss)



        # perform parameter update with Adagrad
        for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                      [dWxh, dWhh, dWhy, dbh, dby], 
                                      [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
        ps = forward_pass(inputs)
        acc = accuracy(ps, targets, correctness_for_student)      
        print ('epoch %d, iter %d, loss: %f, acc: %f' % (e, i, loss, acc)) # print progress
        total_acc += acc
    total_acc /= num_train
    print ('epoch %d, acc: %f' % (e, total_acc)) # print progress
    

epoch 0, iter 0, loss: 125.533393, acc: 0.346939
epoch 0, iter 1, loss: 39.431083, acc: 0.816327
epoch 0, iter 2, loss: 170.097497, acc: 0.306122
epoch 0, iter 3, loss: 163.023807, acc: 0.571429
epoch 0, iter 4, loss: 326.668227, acc: 0.285714
epoch 0, iter 5, loss: 290.488278, acc: 0.387755
epoch 0, iter 6, loss: 324.267898, acc: 0.244898
epoch 0, iter 7, loss: 166.772631, acc: 0.489796
epoch 0, iter 8, loss: 166.257124, acc: 0.653061
epoch 0, iter 9, loss: 270.621781, acc: 0.244898
epoch 0, iter 10, loss: 264.267423, acc: 0.387755
epoch 0, iter 11, loss: 142.655729, acc: 0.632653
epoch 0, iter 12, loss: 194.548714, acc: 0.224490
epoch 0, iter 13, loss: 195.350284, acc: 0.346939
epoch 0, iter 14, loss: 220.886382, acc: 0.306122
epoch 0, iter 15, loss: 233.996487, acc: 0.102041
epoch 0, iter 16, loss: 113.528345, acc: 0.571429
epoch 0, iter 17, loss: 225.665717, acc: 0.224490
epoch 0, iter 18, loss: 61.258655, acc: 0.734694
epoch 0, iter 19, loss: 245.891783, acc: 0.040816
epoch 0, ite

In [ ]:
plt.plot(xrange(len(losses)), losses)

# plot the trend line with error bars that correspond to standard deviation
plt.title('Loss History')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.show()